In [1]:
import tkinter as tk
from tkinter import ttk, filedialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow import keras
import threading
import time

class ASLDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("ASL Detection App")
        self.root.geometry("800x600")
        
        self.model = None
        self.class_names = ['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
                            'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
        
        self.cap = None
        self.is_camera_running = False
        self.detection_active = False
        
        self.create_ui()
        
    def create_ui(self):

        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        control_frame = ttk.Frame(main_frame)
        control_frame.pack(fill=tk.X, pady=10)
        
        load_model_btn = ttk.Button(control_frame, text="Load Model", command=self.load_model)
        load_model_btn.pack(side=tk.LEFT, padx=5)
        
        self.camera_btn = ttk.Button(control_frame, text="Start Camera", command=self.toggle_camera)
        self.camera_btn.pack(side=tk.LEFT, padx=5)
        
        self.detection_btn = ttk.Button(control_frame, text="Start Detection", 
                                        command=self.toggle_detection, state=tk.DISABLED)
        self.detection_btn.pack(side=tk.LEFT, padx=5)
        
        self.status_var = tk.StringVar(value="Status: Ready")
        status_label = ttk.Label(control_frame, textvariable=self.status_var)
        status_label.pack(side=tk.RIGHT, padx=5)
        
        display_frame = ttk.Frame(main_frame)
        display_frame.pack(fill=tk.BOTH, expand=True, pady=10)

        self.video_frame = ttk.Frame(display_frame, borderwidth=2, relief="groove")
        self.video_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)
        
        self.video_label = ttk.Label(self.video_frame)
        self.video_label.pack(fill=tk.BOTH, expand=True)
        
        results_frame = ttk.Frame(display_frame, width=200, borderwidth=2, relief="groove")
        results_frame.pack(side=tk.RIGHT, fill=tk.BOTH, padx=5)
        results_frame.pack_propagate(False)
        
        results_title = ttk.Label(results_frame, text="Detection Results", font=("Arial", 12, "bold"))
        results_title.pack(pady=10)
        
        self.prediction_var = tk.StringVar(value="Letter: -")
        prediction_label = ttk.Label(results_frame, textvariable=self.prediction_var, font=("Arial", 36))
        prediction_label.pack(pady=20)
        
        self.confidence_var = tk.StringVar(value="Confidence: -")
        confidence_label = ttk.Label(results_frame, textvariable=self.confidence_var)
        confidence_label.pack(pady=5)
        
        instructions_frame = ttk.LabelFrame(main_frame, text="Instructions")
        instructions_frame.pack(fill=tk.X, pady=10)
        
        instructions_text = ("1. Load your trained Keras model\n"
                           "2. Start the camera\n"
                           "3. Show ASL signs in the camera view\n"
                           "4. Press 'Start Detection' to begin recognizing signs")
        
        instructions_label = ttk.Label(instructions_frame, text=instructions_text, justify=tk.LEFT)
        instructions_label.pack(padx=10, pady=5)

    def load_model(self):
        """Load the Keras model from a file dialog"""
        file_path = filedialog.askopenfilename(
            title="Select Keras Model",
            filetypes=[("my_model", "*.keras"), ("All Files", "*.*")]
        )
        
        if file_path:
            try:
                self.status_var.set("Status: Loading model...")
                self.root.update()
                
                self.model = keras.models.load_model(file_path)
                self.status_var.set(f"Status: Model loaded successfully")
                
                if self.is_camera_running:
                    self.detection_btn.config(state=tk.NORMAL)
                
            except Exception as e:
                self.status_var.set(f"Status: Error loading model - {str(e)}")
    
    def toggle_camera(self):
        """Start or stop the camera"""
        if self.is_camera_running:
            self.is_camera_running = False
            self.detection_active = False
            if self.cap is not None:
                self.cap.release()
                self.cap = None
            self.camera_btn.config(text="Start Camera")
            self.detection_btn.config(text="Start Detection", state=tk.DISABLED)
            self.video_label.config(image='')
            self.status_var.set("Status: Camera stopped")
        else:
            self.cap = cv2.VideoCapture(0)
            if self.cap.isOpened():
                self.is_camera_running = True
                self.camera_btn.config(text="Stop Camera")
          
                if self.model is not None:
                    self.detection_btn.config(state=tk.NORMAL)
                
                self.status_var.set("Status: Camera started")
      
                threading.Thread(target=self.update_frame, daemon=True).start()
            else:
                self.status_var.set("Status: Failed to open camera")
    
    def toggle_detection(self):
        """Toggle the ASL detection on/off"""
        if self.detection_active:
            self.detection_active = False
            self.detection_btn.config(text="Start Detection")
            self.status_var.set("Status: Detection stopped")
        else:
            self.detection_active = True
            self.detection_btn.config(text="Stop Detection")
            self.status_var.set("Status: Detection active")
    
    def update_frame(self):
        """Update the video frame continuously"""
        while self.is_camera_running:
            ret, frame = self.cap.read()
            if ret:
            
                frame = cv2.flip(frame, 1)
                 
                h, w = frame.shape[:2]
                detection_size = min(h, w) // 2
                x1 = w // 2 - detection_size // 2
                y1 = h // 2 - detection_size // 2
                x2 = x1 + detection_size
                y2 = y1 + detection_size
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                if self.detection_active and self.model is not None:
                    roi = frame[y1:y2, x1:x2]
                    prediction, confidence = self.predict_sign(roi)
                    
                    self.prediction_var.set(f"Letter: {prediction}")
                    self.confidence_var.set(f"Confidence: {confidence:.2f}%")
                    
                    cv2.putText(frame, f"{prediction} ({confidence:.1f}%)", 
                                (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 
                                0.9, (0, 255, 0), 2)
            
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(rgb_frame)
                imgtk = ImageTk.PhotoImage(image=img)

                self.video_label.config(image=imgtk)
                self.video_label.image = imgtk
            
            time.sleep(0.03)  # ~30 FPS
    
    def predict_sign(self, image):
        """Predicts the ASL sign from the image"""
        try:
            # preprocess the image 
            input_size = (128, 128)
            processed_img = cv2.resize(image, input_size)
            processed_img = processed_img / 255.0  
            
            input_data = np.expand_dims(processed_img, axis=0)
            
            predictions = self.model.predict(input_data)
            
            predicted_class_idx = np.argmax(predictions[0])
            confidence = predictions[0][predicted_class_idx] * 100
            
            if predicted_class_idx < len(self.class_names):
                predicted_class = self.class_names[predicted_class_idx]
            else:
                predicted_class = "Unknown"
                
            return predicted_class, confidence
            
        except Exception as e:
            print(f"Prediction error: {str(e)}")
            return "Error", 0.0

def main():
    root = tk.Tk()
    app = ASLDetectionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 508ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 